In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import cv2
import numpy as np
import csv

In [22]:
raw_folder_path = "../../adobe5k/raw"
edited_folder_path = "../../adobe5k/a"
csv_file_path = "adobe5k_a.csv"

In [23]:

# This will form the value for exposure
def calculate_exposure_with_cv2(img):
    # Convert the image from BGR to HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Extract the Saturation channel
    _, _, V = cv2.split(hsv_img)
    # Calculate average saturation
    average_exposure = np.mean(V)
    return (average_exposure/255)


def calculate_saturation_with_cv2(img):
    # Convert the image from BGR to HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Extract the Saturation channel
    _, S, _ = cv2.split(hsv_img)
    # Calculate average saturation
    average_saturation = np.mean(S)
    return (average_saturation/255)

def calculate_hue_with_cv2(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Extract the Saturation channel
    H, _, _ = cv2.split(hsv_img)
    # Calculate average saturation
    average_hue = np.mean(H)
    return (average_hue/180)

def calculate_all_parameters_with_cv2(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv_img)
    average_hue = np.mean(H)
    average_saturation = np.mean(S)
    average_exposure = np.mean(V)
    return (average_hue/180, average_saturation/255, average_exposure/255)

# This will be used for modifying image
def adjust_gamma(image_path, gamma=1.0):
    image=cv2.imread(image_path)
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

In [24]:
#iterater through the images and extract the exposure, saturation and hue 
#make a list of the values and return the list

# def extract_values(folder_path):
#     exposure = []
#     saturation = []
#     hue = []
#     for filename in os.listdir(folder_path):
#         img = cv2.imread(os.path.join(folder_path, filename))
#         my_tuple = calculate_all_parameters_with_cv2(img)
#         exposure.append(my_tuple[2])
#         saturation.append(my_tuple[1])
#         hue.append(my_tuple[0])
#     return exposure, saturation, hue


def extract_values(folder_path):
    exposure = []
    saturation = []
    hue = []
    try:
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    my_tuple = calculate_all_parameters_with_cv2(img)
                    exposure.append(my_tuple[2])
                    saturation.append(my_tuple[1])
                    hue.append(my_tuple[0])
                else:
                    print(f"Skipping {filename} as it cannot be read by OpenCV.")
            else:
                print(f"Skipping {filename} as it is not a file.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    return exposure, saturation, hue




In [25]:
# raw_exposure, raw_saturation, raw_hue = extract_values(raw_folder_path)
# edited_exposure, edited_saturation, edited_hue = extract_values(edited_folder_path)

In [26]:
#write to csv 

# with open(csv_file_path, mode='w') as adobe5k_file:
#     adobe5k_writer = csv.writer(adobe5k_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     adobe5k_writer.writerow(['Raw Exposure', 'Edited Exposure', 'Raw Saturation', 'Edited Saturation', 'Raw Hue', 'Edited Hue'])
#     for i in range(len(raw_exposure)):
#         adobe5k_writer.writerow([raw_exposure[i], edited_exposure[i], raw_saturation[i], edited_saturation[i], raw_hue[i], edited_hue[i]])


In [27]:
data = pd.read_csv(csv_file_path)

X_train = data[['Raw Exposure', 'Raw Saturation', 'Raw Hue']]
y_train = data[['Edited Exposure', 'Edited Saturation', 'Edited Hue']]

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [28]:
def adjust_image(img, new_exposure, new_saturation, new_hue):
    # Convert from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Split into the H, S, and V channels
    h, s, v = cv2.split(hsv)

    # Calculate average values for normalization
    avg_v = np.mean(v) / 255
    avg_s = np.mean(s) / 255
    avg_h = np.mean(h) / 180

    # Calculate adjustment factors
    value_scale = new_exposure / avg_v
    saturation_scale = new_saturation / avg_s
    hue_scale = new_hue / avg_h

    # Adjust values
    v = np.clip(v * value_scale, 0, 255).astype(hsv.dtype)
    s = np.clip(s * saturation_scale, 0, 255).astype(hsv.dtype)
    h = np.clip(h * hue_scale, 0, 180).astype(hsv.dtype)
    
    # Merge the adjusted H, S, and V channels back together
    hsv_adjusted = cv2.merge([h, s, v])
    
    # Convert back from HSV to BGR
    img_adjusted = cv2.cvtColor(hsv_adjusted, cv2.COLOR_HSV2BGR)
    
    return img_adjusted


In [29]:
# test data csv file

test_data = "test_data.csv"
test_folder_path = "../../adobe5k/test"
output_folder = "../../adobe5k/test_a_output"

test_raw_exposure, test_raw_saturation, test_raw_hue = extract_values(test_folder_path)

with open(test_data, mode='w') as test_file:
    test_writer = csv.writer(test_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    test_writer.writerow(['Raw Exposure', 'Raw Saturation', 'Raw Hue'])
    for i in range(len(test_raw_exposure)):
        test_writer.writerow([test_raw_exposure[i], test_raw_saturation[i], test_raw_hue[i]])

test_data = pd.read_csv(test_data)

X_test = test_data[['Raw Exposure', 'Raw Saturation', 'Raw Hue']]

y_pred = model.predict(X_test)

# print(y_pred)



In [30]:
os.makedirs(output_folder, exist_ok=True)

files = os.listdir(test_folder_path)

# print(files)

for i, file in enumerate(files):
    img = cv2.imread(os.path.join(test_folder_path, file))
    if img is None:
        print(f"Failed to read image: {file}")
        continue
    
    img_adjusted = adjust_image(img, y_pred[i][0], y_pred[i][1], y_pred[i][2])
    cv2.imwrite(os.path.join(output_folder, file), img_adjusted)
